# Tratando Dataset

In [1]:
import xml.etree.ElementTree as xet
from glob import glob
import pandas as pd
import numpy as np
import os
import cv2
import tensorflow as tf
from tensorflow.keras.models import Model
from tensorflow.keras.callbacks import TensorBoard
from sklearn.model_selection import train_test_split
from tensorflow.keras.applications import InceptionResNetV2
from tensorflow.keras.layers import Dense, Dropout, Flatten, Input
from tensorflow.keras.preprocessing.image import load_img, img_to_array

2024-04-05 11:57:25.577882: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1


In [2]:
!curl -L -o /tmp/tcc_alpd/archive.tar.gz "https://www.dropbox.com/scl/fi/khnnv0x8pcpbzlgfevqac/archive.tar.gz?rlkey=55w3pkxp02szjpqw22df7hmn8&dl=0"

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100   256    0   256    0     0    241      0 --:--:--  0:00:01 --:--:--   241
100   491    0   491    0     0    216      0 --:--:--  0:00:02 --:--:--   625
100  202M  100  202M    0     0  14.1M      0  0:00:14  0:00:14 --:--:-- 18.8M


In [3]:
!tar xfv /tmp/tcc_alpd/archive.tar.gz -C /tmp/tcc_alpd

archive/
archive/.DS_Store
archive/images/
archive/annotations/
archive/annotations/Cars213.xml
archive/annotations/Cars207.xml
archive/annotations/Cars398.xml
archive/annotations/Cars367.xml
archive/annotations/Cars401.xml
archive/annotations/Cars415.xml
archive/annotations/Cars373.xml
archive/annotations/Cars429.xml
archive/annotations/Cars47.xml
archive/annotations/Cars53.xml
archive/annotations/Cars165.xml
archive/annotations/Cars171.xml
archive/annotations/Cars159.xml
archive/annotations/Cars84.xml
archive/annotations/Cars90.xml
archive/annotations/Cars91.xml
archive/annotations/Cars85.xml
archive/annotations/Cars158.xml
archive/annotations/Cars170.xml
archive/annotations/Cars164.xml
archive/annotations/Cars52.xml
archive/annotations/Cars46.xml
archive/annotations/Cars428.xml
archive/annotations/Cars414.xml
archive/annotations/Cars372.xml
archive/annotations/Cars366.xml
archive/annotations/Cars400.xml
archive/annotations/Cars399.xml
archive/annotations/Cars206.xml
archive/annotati

In [4]:
path = glob('/tmp/tcc_alpd/archive/annotations/*.xml')
labels_dict = dict(filepath=[],xmin=[],xmax=[],ymin=[],ymax=[])
for filename in path:

    info = xet.parse(filename)
    root = info.getroot()
    member_object = root.find('object')
    labels_info = member_object.find('bndbox')
    xmin = int(labels_info.find('xmin').text)
    xmax = int(labels_info.find('xmax').text)
    ymin = int(labels_info.find('ymin').text)
    ymax = int(labels_info.find('ymax').text)

    labels_dict['filepath'].append(filename)
    labels_dict['xmin'].append(xmin)
    labels_dict['xmax'].append(xmax)
    labels_dict['ymin'].append(ymin)
    labels_dict['ymax'].append(ymax)
    

In [5]:
df = pd.DataFrame(labels_dict)
df.to_csv('labels.csv',index=False)
df.head()

,filepath,xmin,xmax,ymin,ymax
0,/tmp/tcc_alpd/archive/annotations/Cars400.xml,303,340,191,217
1,/tmp/tcc_alpd/archive/annotations/Cars104.xml,195,244,266,282
2,/tmp/tcc_alpd/archive/annotations/Cars224.xml,181,243,250,275
3,/tmp/tcc_alpd/archive/annotations/Cars311.xml,295,355,166,185
4,/tmp/tcc_alpd/archive/annotations/Cars319.xml,167,232,228,259


In [6]:
def getFilename(filename):
    filename_image = xet.parse(filename).getroot().find('filename').text
    filepath_image = os.path.join('/tmp/tcc_alpd/archive/images',filename_image)
    return filepath_image

In [7]:
image_path = list(df['filepath'].apply(getFilename))
image_path

['/tmp/tcc_alpd/archive/images/Cars400.png',
 '/tmp/tcc_alpd/archive/images/Cars104.png',
 '/tmp/tcc_alpd/archive/images/Cars224.png',
 '/tmp/tcc_alpd/archive/images/Cars311.png',
 '/tmp/tcc_alpd/archive/images/Cars319.png',
 '/tmp/tcc_alpd/archive/images/Cars73.png',
 '/tmp/tcc_alpd/archive/images/Cars11.png',
 '/tmp/tcc_alpd/archive/images/Cars370.png',
 '/tmp/tcc_alpd/archive/images/Cars326.png',
 '/tmp/tcc_alpd/archive/images/Cars304.png',
 '/tmp/tcc_alpd/archive/images/Cars347.png',
 '/tmp/tcc_alpd/archive/images/Cars367.png',
 '/tmp/tcc_alpd/archive/images/Cars186.png',
 '/tmp/tcc_alpd/archive/images/Cars364.png',
 '/tmp/tcc_alpd/archive/images/Cars299.png',
 '/tmp/tcc_alpd/archive/images/Cars351.png',
 '/tmp/tcc_alpd/archive/images/Cars228.png',
 '/tmp/tcc_alpd/archive/images/Cars346.png',
 '/tmp/tcc_alpd/archive/images/Cars278.png',
 '/tmp/tcc_alpd/archive/images/Cars365.png',
 '/tmp/tcc_alpd/archive/images/Cars129.png',
 '/tmp/tcc_alpd/archive/images/Cars292.png',
 '/tmp/tcc_a

In [8]:
labels = df.iloc[:,1:].values
data = []
output = []
for ind in range(len(image_path)):
    image = image_path[ind]
    img_arr = cv2.imread(image)
    h,w,d = img_arr.shape

    load_image = load_img(image,target_size=(224,224))
    load_image_arr = img_to_array(load_image)
    norm_load_image_arr = load_image_arr/255.0 # Normalization

    xmin,xmax,ymin,ymax = labels[ind]
    nxmin,nxmax = xmin/w,xmax/w
    nymin,nymax = ymin/h,ymax/h
    label_norm = (nxmin,nxmax,nymin,nymax) # Normalized output

    data.append(norm_load_image_arr)
    output.append(label_norm)

In [9]:
X = np.array(data,dtype=np.float32)
y = np.array(output,dtype=np.float32)

In [10]:
x_train,x_test,y_train,y_test = train_test_split(X,y,train_size=0.8,random_state=0)
x_train.shape,x_test.shape,y_train.shape,y_test.shape

((346, 224, 224, 3), (87, 224, 224, 3), (346, 4), (87, 4))

# Modelo

In [11]:
from tensorflow.python.client import device_lib
print(device_lib.list_local_devices())


[name: "/device:CPU:0"
device_type: "CPU"
memory_limit: 268435456
locality {
}
incarnation: 13501974832531469597
, name: "/device:GPU:0"
device_type: "GPU"
memory_limit: 7604932448
locality {
  bus_id: 1
  links {
  }
}
incarnation: 4977690055209845640
physical_device_desc: "device: 0, name: NVIDIA GeForce RTX 2080, pci bus id: 0000:01:00.0, compute capability: 7.5"
]


2024-04-05 11:57:51.972022: I tensorflow/core/platform/cpu_feature_guard.cc:142] This TensorFlow binary is optimized with oneAPI Deep Neural Network Library (oneDNN) to use the following CPU instructions in performance-critical operations:  SSE4.1 SSE4.2 AVX AVX2 FMA
To enable them in other operations, rebuild TensorFlow with the appropriate compiler flags.
2024-04-05 11:57:51.972701: I tensorflow/compiler/jit/xla_gpu_device.cc:99] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-04-05 11:57:51.973733: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcuda.so.1
2024-04-05 11:57:52.026710: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-05 11:57:52.026857: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 na

In [12]:
inception_resnet = InceptionResNetV2(weights="imagenet",include_top=False, input_tensor=Input(shape=(224,224,3)))
# ---------------------
headmodel = inception_resnet.output
headmodel = Flatten()(headmodel)
headmodel = Dense(500,activation="relu")(headmodel)
headmodel = Dense(250,activation="relu")(headmodel)
headmodel = Dense(4,activation='sigmoid')(headmodel)


# ---------- model
model = Model(inputs=inception_resnet.input,outputs=headmodel)

2024-04-05 11:57:52.631055: I tensorflow/compiler/jit/xla_cpu_device.cc:41] Not creating XLA devices, tf_xla_enable_xla_devices not set
2024-04-05 11:57:52.631234: I tensorflow/stream_executor/cuda/cuda_gpu_executor.cc:941] successful NUMA node read from SysFS had negative value (-1), but there must be at least one NUMA node, so returning NUMA node zero
2024-04-05 11:57:52.631367: I tensorflow/core/common_runtime/gpu/gpu_device.cc:1720] Found device 0 with properties: 
pciBusID: 0000:01:00.0 name: NVIDIA GeForce RTX 2080 computeCapability: 7.5
coreClock: 1.71GHz coreCount: 46 deviceMemorySize: 7.78GiB deviceMemoryBandwidth: 417.23GiB/s
2024-04-05 11:57:52.631389: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudart.so.10.1
2024-04-05 11:57:52.631420: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-04-05 11:57:52.631436: I tensorflow/stream_executor/platform/

In [13]:
model.compile(loss='mse',optimizer=tf.keras.optimizers.Adam(learning_rate=1e-4))
model.summary()

Model: "model"
__________________________________________________________________________________________________
Layer (type)                    Output Shape         Param #     Connected to                     
input_1 (InputLayer)            [(None, 224, 224, 3) 0                                            
__________________________________________________________________________________________________
conv2d (Conv2D)                 (None, 111, 111, 32) 864         input_1[0][0]                    
__________________________________________________________________________________________________
batch_normalization (BatchNorma (None, 111, 111, 32) 96          conv2d[0][0]                     
__________________________________________________________________________________________________
activation (Activation)         (None, 111, 111, 32) 0           batch_normalization[0][0]        
______________________________________________________________________________________________

In [14]:
tfb = TensorBoard('object_detection')
history = model.fit(x=x_train,y=y_train,batch_size=10,epochs=50,
                    validation_data=(x_test,y_test),callbacks=[tfb])

2024-04-05 11:57:56.133180: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2024-04-05 11:57:56.133200: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2024-04-05 11:57:56.133228: I tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1365] Profiler found 1 GPUs
2024-04-05 11:57:56.133936: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcupti.so.10.1
2024-04-05 11:57:56.234527: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2024-04-05 11:57:56.235979: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2024-04-05 11:57:56.496028: I tensorflow/compiler/mlir/mlir_graph_optimization_pass.cc:116] None of the MLIR optimization passes are enabled (registered 2)
2024-04-05 11:57

Epoch 1/50


2024-04-05 11:58:06.698264: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcublas.so.10
2024-04-05 11:58:06.939608: I tensorflow/stream_executor/platform/default/dso_loader.cc:49] Successfully opened dynamic library libcudnn.so.7


 2/35 [>.............................] - ETA: 12s - loss: 0.0825 

2024-04-05 11:58:09.113995: I tensorflow/core/profiler/lib/profiler_session.cc:136] Profiler session initializing.
2024-04-05 11:58:09.114020: I tensorflow/core/profiler/lib/profiler_session.cc:155] Profiler session started.
2024-04-05 11:58:09.114665: E tensorflow/core/profiler/internal/gpu/cupti_tracer.cc:1415] function cupti_interface_->Subscribe( &subscriber_, (CUpti_CallbackFunc)ApiCallback, this)failed with error CUPTI_ERROR_INSUFFICIENT_PRIVILEGES
2024-04-05 11:58:09.245617: I tensorflow/core/profiler/lib/profiler_session.cc:71] Profiler session collecting data.


 3/35 [=>............................] - ETA: 9s - loss: 0.0825 

2024-04-05 11:58:09.275295: I tensorflow/core/profiler/internal/gpu/cupti_collector.cc:228]  GpuTracer has collected 0 callback api events and 0 activity events. 
2024-04-05 11:58:09.283185: I tensorflow/core/profiler/lib/profiler_session.cc:172] Profiler session tear down.
2024-04-05 11:58:09.290017: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: object_detection/train/plugins/profile/2024_04_05_11_58_09
2024-04-05 11:58:09.292527: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for trace.json.gz to object_detection/train/plugins/profile/2024_04_05_11_58_09/alertrack.trace.json.gz
2024-04-05 11:58:09.328590: I tensorflow/core/profiler/rpc/client/save_profile.cc:137] Creating directory: object_detection/train/plugins/profile/2024_04_05_11_58_09
2024-04-05 11:58:09.333913: I tensorflow/core/profiler/rpc/client/save_profile.cc:143] Dumped gzipped tool data for memory_profile.json.gz to object_detection/train/plugins/prof

35/35 [==============================] - 20s 236ms/step - loss: 0.0548 - val_loss: 0.0312
Epoch 2/50
35/35 [==============================] - 5s 139ms/step - loss: 0.0141 - val_loss: 0.0187
Epoch 3/50
35/35 [==============================] - 5s 139ms/step - loss: 0.0068 - val_loss: 0.0130
Epoch 4/50
35/35 [==============================] - 5s 139ms/step - loss: 0.0051 - val_loss: 0.0122
Epoch 5/50
35/35 [==============================] - 5s 139ms/step - loss: 0.0035 - val_loss: 0.0114
Epoch 6/50
35/35 [==============================] - 5s 139ms/step - loss: 0.0043 - val_loss: 0.0101
Epoch 7/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0026 - val_loss: 0.0091
Epoch 8/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0026 - val_loss: 0.0090
Epoch 9/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0023 - val_loss: 0.0087
Epoch 10/50
35/35 [==============================] - 5s 140ms/step - loss: 0.0021 - val_loss: 0.0083
Epoch 11/

In [16]:
model.save('./license_plate_detection.h5')